In [ ]:
# approach the object
import rospy
from geometry_msgs.msg import Twist
import actionlib
from control_msgs.msg import FollowJointTrajectoryAction, FollowJointTrajectoryGoal
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
from rail_segmentation.srv import *
from rail_manipulation_msgs.srv import *
import tf2_ros
import numpy as np
from sensor_msgs.msg import PointCloud2
import sensor_msgs.point_cloud2 as pc2
from ros_numpy import numpify, msgify

In [ ]:
def convert_pointcloud2_to_pc(pointcloud2_msg):
    pc_data = pc2.read_points(pointcloud2_msg, skip_nans=True, field_names=("x", "y", "z"))
    pc_list = []

    for p in pc_data:
        pc_list.append([p[0], p[1], p[2]])

    pc_array = np.array(pc_list, dtype=np.float32)

    return pc_array

In [ ]:
rospy.init_node('command_sender', anonymous=True)
base_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=10)
rate = rospy.Rate(10)  # Adjust the frequency as per your requirement

def move_base(base_pub_, linear_vel, angular_vel):
    cmd_vel = Twist()
    cmd_vel.linear.x = linear_vel
    cmd_vel.angular.z = angular_vel
    base_pub_.publish(cmd_vel)

In [ ]:
torso_controller_client = actionlib.SimpleActionClient('/torso_controller/follow_joint_trajectory',
                                          FollowJointTrajectoryAction)
torso_controller_client.wait_for_server()
# control the head
head_controller_client = actionlib.SimpleActionClient(
            "/head_controller/follow_joint_trajectory",
            FollowJointTrajectoryAction,
        )
head_controller_client.wait_for_server()
# detect the object
# get camera transform from tf tree
tfBuffer = tf2_ros.Buffer()
listener = tf2_ros.TransformListener(tfBuffer)

In [ ]:
# move the head toward the object
def look_at_object(head_controller_client_, tfBuffer_, runtime = 1.0):
    try:
        obj_in_base_trans = tfBuffer_.lookup_transform('base_link', 'can', rospy.Time())
    except (tf2_ros.LookupException, tf2_ros.ConnectivityException, tf2_ros.ExtrapolationException):
        print("tf error")
        
    try:
        head_pan_in_base_trans = tfBuffer_.lookup_transform('base_link', 'head_pan_link', rospy.Time())
    except (tf2_ros.LookupException, tf2_ros.ConnectivityException, tf2_ros.ExtrapolationException):
        print("tf error")
    
    object_in_head = np.array([obj_in_base_trans.transform.translation.x - head_pan_in_base_trans.transform.translation.x,
                               obj_in_base_trans.transform.translation.y - head_pan_in_base_trans.transform.translation.y,
                               obj_in_base_trans.transform.translation.z - head_pan_in_base_trans.transform.translation.z])

    msg = FollowJointTrajectoryGoal()
    msg.trajectory.header.frame_id = ''
    msg.trajectory.joint_names = ['head_pan_joint', 'head_tilt_joint']

    point = JointTrajectoryPoint()
    point.positions = [np.arctan2(object_in_head[1] , object_in_head[0]),
                       -np.arctan2(object_in_head[2] , object_in_head[0])]
    point.time_from_start = rospy.Duration(runtime)

    msg.trajectory.header.stamp = rospy.Time.now()
    msg.trajectory.points = [point]

    head_controller_client_.send_goal_and_wait(msg, execute_timeout=rospy.Duration(runtime))

# # print hand_pan_error

In [ ]:
# lift up the head
look_at_object(head_controller_client, tfBuffer)

In [ ]:
# find the table first
rospy.wait_for_service('table_searcher/search_table')
table_searcher = rospy.ServiceProxy('table_searcher/search_table', SearchTable)

In [ ]:
table_info = table_searcher()

In [ ]:
# adjust the torso height based on the table.
msg = FollowJointTrajectoryGoal()
msg.trajectory.header.frame_id = ''
msg.trajectory.joint_names = ['torso_lift_joint']

point = JointTrajectoryPoint()
point.positions = [table_info.center.z - 0.5]
point.time_from_start = rospy.Duration(1.0)

msg.trajectory.header.stamp = rospy.Time.now()
msg.trajectory.points = []
msg.trajectory.points.append(point)

torso_controller_client.send_goal_and_wait(
    msg, execute_timeout=rospy.Duration(2.0)
)

look_at_object(head_controller_client, tfBuffer)


In [ ]:
# first, rotate the robot forward to the object.
can_trans = tfBuffer.lookup_transform('base_link', 'can', rospy.Time())
rot_error = np.arctan2(can_trans.transform.translation.y, can_trans.transform.translation.x)
while abs(rot_error) > 0.02:
    move_base(base_pub, 0.0, rot_error)
    try:
        can_trans = tfBuffer.lookup_transform('base_link', 'can', rospy.Time())
    except (tf2_ros.LookupException, tf2_ros.ConnectivityException, tf2_ros.ExtrapolationException):
        print("tf error")
    rot_error = np.arctan2(can_trans.transform.translation.y, can_trans.transform.translation.x)

look_at_object(head_controller_client, tfBuffer)

In [ ]:
def min_dist_from_pointcloud_to_robot(camera_trans, pc):
    camera_pose = numpify(camera_trans.transform)
    current_pc = convert_pointcloud2_to_pc(pc)
    homogeneous_point_cloud = np.hstack([current_pc, np.ones((current_pc.shape[0], 1))])
    transformed_point_cloud = np.dot(camera_pose, homogeneous_point_cloud.T).T
    transformed_point_cloud = transformed_point_cloud[:, :3]
    filtered_current_pc = transformed_point_cloud[transformed_point_cloud[:,-1] > 0.05]
    return np.linalg.norm(filtered_current_pc, axis=1, keepdims=True).min()

In [ ]:
# look down for detect table is comming.
msg = FollowJointTrajectoryGoal()
msg.trajectory.header.frame_id = ''
msg.trajectory.joint_names = ['head_pan_joint', 'head_tilt_joint']

point = JointTrajectoryPoint()
point.positions = [0.0, 0.88]
point.time_from_start = rospy.Duration(1.0)

msg.trajectory.header.stamp = rospy.Time.now()
msg.trajectory.points = [point]

head_controller_client.send_goal_and_wait(msg, execute_timeout=rospy.Duration(1.0))


In [ ]:
try:
    camera_trans = tfBuffer.lookup_transform('base_link', 'head_camera_rgb_optical_frame', rospy.Time())
except (tf2_ros.LookupException, tf2_ros.ConnectivityException, tf2_ros.ExtrapolationException):
    print("tf error")
    
data = rospy.wait_for_message("/head_camera/depth_downsample/points", PointCloud2)
dist_to_table = min_dist_from_pointcloud_to_robot(camera_trans, data)
while dist_to_table > 0.8:
    move_base(base_pub, dist_to_table * 0.5, 0.0)
    data = rospy.wait_for_message("/head_camera/depth_downsample/points", PointCloud2)
    dist_to_table = min_dist_from_pointcloud_to_robot(camera_trans, data)
    print dist_to_table
    
look_at_object(head_controller_client, tfBuffer)